# Improving Machine Learning Experience With Multimedia Techniques
Marco A. Franchi

## *Abstract*

*It has been very common face machine learning algorithms at the multimedia area: traffic count; real-time vigilance cameras; baggage tracker; face/expression recognition; and so on. It is so common that it was designed a name for it, Machine Vision. However, it is perceptive the gap between machine learning and multimedia solutions, where even at simple embedded systems it is possible to reach 4k videos running at 60 frames per second, whereas the best neural network solution only handles 224x224 frames at 300 milliseconds in the same system. Due to this gap, a vast number of solutions as being developed: dedicated hardware for inference process; model manipulation; accelerated pre-processing image solutions; and video manipulation techniques. This paper is based at the study of these videos manipulation techniques, exposing the most common algorithmics, such as frame-skip, frame-droop, resizing, color convert, and overlay solutions; and showing the main difference between them, when the solution can be applied, and the expected performance for each one.*



## Introduction

Aiming to diminish the gap between the machine learning inference process and the multimedia capability, which reaches 4k@60fps, some video manipulation solutions was purposed. Among them, the most common is the overlay solutions, which are able to create alpha layers over the video and insert information on it. These overlays are very common on object detections algorithmics, once they are responsible for drawing a square, select or color an object at the scene. The most common overlays are Scalable Vector Graphics (SVG), Cairo, and OpenCV.
In addition to the overlays, it is important to use a great video framework able to handle all the elements involved at the inference and display solutions. One of the best and most useful ones is the GStreamer framework. GStreamer is able to handle plugins in pipelines, which is perfect to do quick tests.
Apart from the video solutions, it is important to choose the best machine learning algorithmics as well. With a focus on object detection, the most common and valued ones are Single Shot Detection (SSD) and Tensorflow. Both have an incredible inference process capability and the TFlite version demonstrated a great tool for embedded systems.
Thus, with all the tools chosen, this paper intends to compare the combination of theses algorithmics for object detection solutions. This comparison aims to demonstrate how we can increase the video frame rate with simple approaches and demonstrate the best scenarios to handle each neural network algorithmics and the overlays plugins behavior on these tests.

## Materials and Methods

This section describes the material such as the video files, models, labels, and programming language used, and the adopted methodology.

### Programming Language

This paper uses Python 3 language and all the required support for the object detection algorithm: the overlays libs, GStreamer plugins, Tensorflow Lite, and SSD.
* **OpenCV**: Open Source Computer Vision Library, this library is cross-platform and free for user under open-source BSD license. 
* **SVG image**: Scalable Vector Graphics, a Extensible Markup Language (XML)-based vector image format for two-dimensional graphics.  
* **Cairo**: Cairo is a 2D graphics library with support for multiples output devices and is available under LGPLv2.1 license.
* **GStreamer**: GStreamer is a library for constructiong graphics of media-handling components and is released under the LGPL license.
* **TFLite**: Tensoflow is a library for dataflow and differenctiable programming across of tasks, free and open-source under the Apache License 2.0. TFLite is a special version for mobile development. 
* **SSD**: Single Shot Detector algorithm designed for object detection in real-time applications.

### Models and Labels

As the focus of this paper is the video techniques, and as the SSD and TFlite already have a huge numbers of pre-processed models, this paper will not care about pre-processing or training models and will uses pre-processed models available at the TFLite official object detection support web page.

For this, the following pre-tested model and labels will be used at the tests:

**Model**: mobilenet_ssd_v2_coco_quant_postprocess.tflite

**Labels**: coco_labels.txt

### GStreamer and V4L2

As mentioned before, this paper uses GStreamer framework to reproduce the videos files.
For the comparison purpose, the following approaches will be performed:
* OpenCV V4L2 directly handle;
* GStreamer appsink pipeline + OpenCV V4L2 output;
* GStreamer appsink + appsrc pipelines;
* GStreamer overlay plugins support.

The workflow below describes the difference between each process:

### Workflow

**OpenCV V4L2 directly handle:**
![opencv](../data/images/opencv_v4l2.png)
For this approach, the OpenCV needs to handle the entire process: open the video, get the frame, resize and do a color convert on it, draw the informations at the frame, and display the results.
This is easely to test, but not fast for ML purpose, once the entire process will be depent from OpenCV.

The following code demonstrate the OpenCV V4L2 use case:

In [ ]:
from ObjectDetection import ObjectsDetectionOpenCV


def main():
    app = ObjectsDetectionOpenCV()
    app.run()


if __name__ == '__main__':
    main()

**GStreamer appsink pipeline + OpenCV V4L2 output**
![appsink](../data/images/appsink_opencv_v4l2.png)
This idea shows a better perfomance than OpenCV V4L2 due the GStremaer usage at the video openning process. It represents one less action to be processed by the OpenCV, and the GStreamer pipelines enables a vast number of plugins and its properties, which can guarantee some aditional chacateristics, such as dropping frame property.
It means that the displayed frame rate will not be impacted for the inference process time. However, the videoconvert usage, required for the appsink to be able to display the results at screen, is a disavantage, once its only supported by CPU, and resize/color convert by CPU has a high processing coast.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionV4l2


def main():
    app = ObjectsDetectionV4l2()
    app.run()


if __name__ == '__main__':
    main()

**GStreamer appsink + appsrc pipelines**
![appsink/appsrc](../data/images/appsink_appsrc.png)
This combination shows very promissor, once we can use the appsink dropping frame property, but do not requires videoconvert, once its results will not be displayed yet. Actually, the resulted data will be processed by the inference process, which can include resizing and color convert, and the results will be send again to GStreamer, only to handle it and display at the screen.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionOpenCV


def main():
    app = ObjectsDetectionOpenCV()
    app.run()


if __name__ == '__main__':
    main()

**GStreamer overlay plugins support:**
![overlay](../data/images/gstreamer_overlay.png)

This is the best approach, but the most dificult to use. Here, the tee usage create two threads: one to be processed by the inference process; other to be displayed. The videobox keeps the frame and is able to return it ot the overlay plugin, so what we see is the combination of two frames, one is the original video, without be touched, other is an alpha image with all the required resizing process being displayed over the original video.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionOpenCV


def main():
    app = ObjectsDetectionOpenCV()
    app.run()


if __name__ == '__main__':
    main()

### Overlays

The last modularized GStreamer code as the overlay property. This overlay will be used to display the following at the scree:
* inference time
* video FPS (Frame per second)
* draw a square around the detected objects
* put a label at these objects
* confidence of it

One example is displayed at the image below:

![detection](../data/images/car_detection.gif)

This overlays procces comparison will be very important for this paper results. So three overlays will be compared:
* Opencv
* Cairo
* SVG

We will use the source code below to use each one:

In [ ]:
def generate_opencv(self, opencv_im, objs, labels):
    height, width, channels = opencv_im.shape
    for obj in objs:
        x0, y0, x1, y1 = list(obj.bbox)
        x0, y0, x1, y1 = int(
            x0 * width), int(
            y0 * height), int(x1 * width), int(y1 * height)

        percent = int(100 * obj.score)
        label = '{}% {}'.format(percent, labels.get(obj.id, obj.id))

        opencv_im = opencv.rectangle(
            opencv_im, (x0, y0), (x1, y1), (0, 255, 0), 2)
        opencv_im = opencv.putText(opencv_im, label, (x0, y0 + 30),
                                   opencv.FONT_HERSHEY_SIMPLEX, 1.0,
                                   (255, 0, 0), 2)
        
def generate_cairo(self, img, objs, labels):
    stride = cairo.ImageSurface.format_stride_for_width(cairo.FORMAT_RGB24, width)
    surface = cairo.ImageSurface.create_for_data(buffer, cairo.FORMAT_RGB24,
                                                 width, height, stride)
    context = cairo.Context(surface)
    overlay = self.overlay()
    x = width - overlay.get_width()
    y = height - overlay.get_height()

    context.set_source_surface(self.overlay(), x, y)
    context.paint()

def generate_svg(self, src_size, inference_size,
                 inference_box, objs, labels, text_lines):
    dwg = svgwrite.Drawing('', size=src_size)
    src_w, src_h = src_size
    inf_w, inf_h = inference_size
    box_x, box_y, box_w, box_h = inference_box
    scale_x, scale_y = src_w / box_w, src_h / box_h

    for y, line in enumerate(text_lines, start=1):
        self.shadow_text(dwg, 10, y * 20, line)
    for obj in objs:
        x0, y0, x1, y1 = list(obj.bbox)
        x, y, w, h = x0, y0, x1 - x0, y1 - y0
        x, y, w, h = int(x * inf_w), int(y *
                                        inf_h), int(w * inf_w), int(h * inf_h)
        x, y = x - box_x, y - box_y
        x, y, w, h = x * scale_x, y * scale_y, w * scale_x, h * scale_y
        percent = int(100 * obj.score)
        label = '{}% {}'.format(percent, labels.get(obj.id, obj.id))
        self.shadow_text(dwg, x, y - 5, label)
        dwg.add(dwg.rect(insert=(x, y), size=(w, h),
                        fill='none', stroke='red', stroke_width='2'))
    return dwg.tostring()
                        sink_element=SINK_ELEMENT, scale_caps=scale_caps)

## Results

The following results.

## Conclusion

This is the conclusion:

## Source

Hwang, C. & Yoon, K., 1981. Multiple Attributes Decision Making Methods and Applications. Berlin: Springer-Verlag.